In [1]:
import cfbd
from cfbd.rest import ApiException
import pandas as pd
import sqlite3

In [27]:
def extract_team_data(year,week):
    """
    Extracts team statistics from a list of game data and converts it into a DataFrame.
    
    This function processes each game in the provided list, extracts information about each team,
    including their IDs, names, home/away status, points scored, and other statistical categories.
    The resulting data is then normalized into a DataFrame for easier analysis.
    
    :param game_data: List of game objects, where each game object contains details about the game
                      and its participating teams.
    :return: DataFrame containing extracted team statistics for each game.
    """
    game_stats=[] # Initialize a list to store statistics for each team
   
    game_data = api_instance.get_team_game_stats(year=year, week=week)
    for game in game_data:
        game_id=game.id # Extract the unique identifier for the game
        teams=game.teams # Get the list of teams involved in the game
        
        # Iterate through each team in the game
        for team in teams:
            team_id = team.school_id # Extract the unique identifier for the team
            team_name = team.school # Extract the name of the team
            home_away = team.home_away # Determine if the team is playing at home or away

            team_stats = {
                'game_id': game_id,
                'team_id': team_id,
                'team_name': team_name,
                'home_away': home_away,
                'points': team.points # Extract the points scored by the team
                }
                # Add stats to the dictionary
            for stat in team.stats:
                category = stat.category
                value = stat.stat
                team_stats[category] = value
                # Append the team's stats to the list of game statistics
            game_stats.append(team_stats)
    # Concatenate the weekly data into the final DataFrame
    return pd.json_normalize(game_stats)

In [47]:
# api_instance.get_team_game_stats(year=2023, week=1)

In [45]:
con=sqlite3.connect('collegeFootball.db')
cursor_obj= con.cursor()
cursor_obj.execute("DROP TABLE IF EXISTS teamStats23")
allWks.to_sql(name='teamStats23',con=con)
con.commit()
con.close()

In [28]:
# allWks=pd.concat([extract_team_data(year=2023,week=i) for i in range(1,2)])

In [5]:
configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = api_key
configuration.api_key_prefix['Authorization']='Bearer'
api_instance = cfbd.GamesApi(cfbd.ApiClient(configuration))

In [12]:
con=sqlite3.connect('collegeFootball.db')
cursor_obj= con.cursor()
cursor_obj.execute("DROP TABLE IF EXISTS teamStats23")

,game_id,team_id,team_name,home_away,points,rushingTDs,passingTDs,kickReturnYards,kickReturnTDs,kickReturns,...,firstDowns,puntReturnYards,puntReturnTDs,puntReturns,tacklesForLoss,defensiveTDs,tackles,sacks,qbHurries,passesDeflected
0,401540244,2320,Lamar,home,17,1,1,82,0,4,...,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,401540244,70,Idaho,away,42,2,4,0,0,1,...,20,26,0,3,NaN,NaN,NaN,NaN,NaN,NaN
2,401525434,2426,Navy,away,3,0,0,12,0,1,...,12,NaN,NaN,NaN,2,0,42,0,1,2
3,401525434,87,Notre Dame,home,42,2,4,41,0,2,...,27,11,0,1,5,0,41,2,2,0
4,401525435,2634,Tennessee State,away,3,0,0,76,0,2,...,12,NaN,NaN,NaN,6,0,45,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,401539480,130,Michigan,away,26,2,0,NaN,NaN,NaN,...,12,87,0,1,5,0,27,4,5,8
18,401539476,166,New Mexico State,away,35,1,4,44,0,3,...,28,NaN,NaN,NaN,1,0,37,0,1,1
19,401539476,2335,Liberty,home,49,5,2,78,0,4,...,34,1,0,1,3,0,30,0,3,7
0,401520445,2426,Navy,away,11,0,1,39,0,1,...,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
p3=pd.read_sql('SELECT * FROM teamStats23;',con)

In [26]:
p3.loc[p3.team_name=='Ohio State']

,index,game_id,team_id,team_name,home_away,points,rushingTDs,passingTDs,kickReturnYards,kickReturnTDs,...,firstDowns,puntReturnYards,puntReturnTDs,puntReturns,tacklesForLoss,defensiveTDs,tackles,sacks,qbHurries,passesDeflected
245,245,401520156,194,Ohio State,away,23,2,0,23,0,...,22,14,0,2,6,0,34,1,0,4
489,229,401520208,194,Ohio State,home,35,2,3,40,0,...,23,3,0,2,3,0,36,2,0,2
598,94,401520240,194,Ohio State,home,63,3,4,40,0,...,24,18,0,1,8,1,38,2,2,6
746,2,401521330,194,Ohio State,away,17,2,0,32,0,...,19,8,0,1,2,0,36,0,3,5
1375,161,401520307,194,Ohio State,home,37,1,2,24,0,...,19,-3,0,3,5,1,36,2,5,5
1597,179,401520323,194,Ohio State,away,41,2,4,49,0,...,24,0,0,1,9,0,35,3,1,5
1805,169,401520343,194,Ohio State,home,20,1,1,14,0,...,22,16,0,4,6,0,31,4,4,6
2035,183,401520360,194,Ohio State,away,24,1,2,None,None,...,23,6,0,1,4,0,27,2,1,4
2275,197,401520375,194,Ohio State,away,35,1,3,-5,0,...,15,None,None,None,4,1,21,1,0,4
2527,201,401520393,194,Ohio State,home,38,2,3,None,None,...,25,11,0,1,6,0,30,2,0,5


In [32]:
p3.loc[p3.game_id==401540244]

,index,game_id,team_id,team_name,home_away,points,rushingTDs,passingTDs,kickReturnYards,kickReturnTDs,...,firstDowns,puntReturnYards,puntReturnTDs,puntReturns,tacklesForLoss,defensiveTDs,tackles,sacks,qbHurries,passesDeflected
0,0,401540244,2320,Lamar,home,17,1,1,82,0,...,16,None,None,None,None,None,None,None,None,None
1,1,401540244,70,Idaho,away,42,2,4,0,0,...,20,26,0,3,None,None,None,None,None,None
